# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [3]:
from sklearn.externals import joblib
eclf = joblib.load('sentiment.pkl') 

IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [4]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [5]:
credentials = {
      "apikey": "private information",
      "iam_apikey_description": "private information",
      "iam_apikey_name": "private information",
      "iam_role_crn": "private information",
      "iam_serviceid_crn": "private information",
      "instance_id": "private information",
      "url": "private information"
  
}

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [6]:
client = WatsonMachineLearningAPIClient(credentials)

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [7]:
metadata = { 
            client.repository.ModelMetaNames.NAME: 'Sistema_de_recomendacion',
            client.repository.ModelMetaNames.RUNTIME_NAME: 'python',
            client.repository.ModelMetaNames.RUNTIME_VERSION: '3.6',
            client.repository.ModelMetaNames.FRAMEWORK_NAME: 'scikit-learn',
            client.repository.ModelMetaNames.AUTHOR_NAME: 'de_Souza_Luan',
}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

tfid_vec = TfidfVectorizer()

pipeline = make_pipeline(tfid_vec, eclf)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth..., random_state=None))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [9]:
online_model = client.repository.store_model(
                                                model= pipeline,
                                                meta_props = metadata,
                                                training_data = X_train,
                                                training_target = y_train
)

**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [10]:
models_details = client.repository.list_models()

------------------------------------  ------------------------  ------------------------  -----------------
GUID                                  NAME                      CREATED                   FRAMEWORK
c01b412e-a0ee-4a45-8e46-fa4b29446338  Sistema_de_recomendacion  2019-10-13T01:17:42.169Z  scikit-learn-0.20
8c4151d7-43a9-4469-8dd5-b41c844bddb7  Sistema_de_recomendacion  2019-10-13T01:01:39.217Z  scikit-learn-0.20
------------------------------------  ------------------------  ------------------------  -----------------


In [11]:
guid = client.repository.get_model_uid(online_model)

**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [12]:
model = client.repository.load(guid)

predictions = model.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [13]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.82      0.92      0.87       200
           1       0.90      0.81      0.85       200

   micro avg       0.86      0.86      0.86       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.86      0.86      0.86       400

